In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip list

Package               Version
--------------------- -----------
altair                4.2.0
argon2-cffi           21.3.0
argon2-cffi-bindings  21.2.0
asttokens             2.0.8
attrs                 22.1.0
autoviz               0.1.43
backcall              0.2.0
beautifulsoup4        4.11.1
bleach                5.0.1
bokeh                 2.4.3
certifi               2022.6.15
cffi                  1.15.1
charset-normalizer    2.1.1
click                 8.1.3
colorama              0.4.5
colorcet              3.0.0
cycler                0.11.0
debugpy               1.6.3
decorator             5.1.1
defusedxml            0.7.1
emoji                 2.0.0
entrypoints           0.4
executing             0.10.0
fastjsonschema        2.16.1
fonttools             4.37.0
fsspec                2022.7.1
holoviews             1.14.9
htmlmin               0.1.12
hvplot                0.8.0
idna                  3.3
ImageHash             4.2.1
importlib-metadata    4.12.0
ipykernel             6.


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Libraries

In [3]:
# import sys
# print(sys.path)
# sys.path.insert(0, "C:\\Users\\KonuTech\\gfk_data_analyst\\gfk_ml_ops_data_analyst\\scripts\\python\\")

In [4]:
import os
# import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import math
from pandas_profiling import ProfileReport
# import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scripts.python.get_metrics import *
from scripts.python.get_charts import *

IndentationError: unexpected indent (get_charts.py, line 14)

In [ ]:
INPUT_PATH="input"
OUTPUT_PATH="output"
CONFIG_PATH="config"

INPUT_DATA="test_data2"
INPUT_DATA_CONFIG="input_config"
OUTPUT_DATA="output"

INPUT_EXTENSION="csv"
INPUT_CONFIG_EXTENSION="json"
OUTPUT_EXTENSION="csv"

INPUT_FILE=f"{INPUT_DATA}.{INPUT_EXTENSION}"
INPUT_CONFIG_FILE=f"{INPUT_DATA_CONFIG}.{INPUT_CONFIG_EXTENSION}"
OUTPUT_FILE=f"{OUTPUT_DATA}.{OUTPUT_EXTENSION}"

INPUT_ABS_APTH=os.path.abspath(os.path.join(INPUT_PATH, INPUT_FILE))
INPUT_FILE_CONFIG=os.path.abspath(os.path.join(CONFIG_PATH, INPUT_CONFIG_FILE))
OUTPUT_ABS_APTH=os.path.abspath(os.path.join(OUTPUT_PATH, OUTPUT_FILE))

In [ ]:
with open(INPUT_FILE_CONFIG, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [ ]:
CONFIG

# Load data

In [ ]:
# LOAD CSV
df = pd.read_csv(
    INPUT_ABS_APTH,
#     index_col=CONFIG['INPUTS']['INDEXES'],
    sep=CONFIG['INPUTS']['SEPARATOR'],
    # quotechar='"',
    # thousands=',',
#     decimal=CONFIG['INPUTS']['DECIMAL'],
    encoding=CONFIG['INPUTS']['ENCODING'],
    infer_datetime_format=True,
    parse_dates=CONFIG['INPUTS']['DATE_COLUMNS'],
    engine="c",
    low_memory=False,
#     float_precision=CONFIG['INPUTS']['DATE_COLUMNS'],
    skipinitialspace=True,
    dtype=CONFIG['INPUTS']['DTYPE']
)

In [ ]:
df.info()

In [ ]:
df.head(10)

In [ ]:
df.describe(
    include='all',
    datetime_is_numeric=True
)

# Pandas data profiler

In [ ]:
profile = ProfileReport(
    df,
    title="Pandas Profiling Report",
    pool_size=0,
    infer_dtypes=False
)

In [ ]:
profile.to_widgets()

### Offline example of Pandas Profiler widget:

![image info](./docs/images/profiler_example.png)

In [ ]:
for col in df.columns:
    if col == 'prod_gr_id':

        get_stacked_bar_chart(
                data_frame=df,
                date_column="period_end_date",
                column_to_group_by=col,
                column_to_count='predict_automatch'
            )

### Dla 'country_id_n' nie działa bo macierze nie są równe ;<
> Nie było zdarzeń z 1 dla 22 krajów

In [ ]:
# for col in df.columns:
#     if col == 'country_id_n':

#         get_stacked_bar_chart(
#                 data_frame=df,
#                 date_column="period_end_date",
#                 column_to_group_by=col,
#                 column_to_count='predict_automatch'
#             )

# Post-Processing Bias Metrics for the Trained Model

## Difference in positive proportion in predicted labels (DPPL)

![image info](./docs/images/DPPL.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)


### CHARACTERISTICS OF METRIC:
> DPPPL > -1 AND DPPPL < 1

> For example, if the model grants loans to 50% of class 2 and to 60% of class 1, then it may be biased against class 2.
We would have to decide whether a 10% difference is material.

## VARIABLE: prod_gr_id

### prod_gr_id total count

In [ ]:
total_counts = df.groupby(['prod_gr_id']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

total_counts

### prod_gr_id by predicted classes

In [ ]:
grouped_counts = df.groupby(['prod_gr_id', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

grouped_counts

In [ ]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == 1]

positive_grouped_counts

In [ ]:
merged_counts = pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

merged_counts

### (DPPL) for prod_gr_id

In [ ]:
ppl_prod_gr_id = get_PPL(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count='predict_automatch'
)

ppl_prod_gr_id

In [ ]:
dppl_prod_gr_id = get_DPPL(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count='predict_automatch'
)

dppl_prod_gr_id

#### Bucketizing dppl_prod_gr_id

In [ ]:
print(pd.cut(dppl_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> Basing solely on **DPPL** we can loosely assume that there is no disparity impact among **prod_gr_id** Classes

## VARIABLE: country_id_n

### country_id_n total count

In [ ]:
total_counts = df.groupby(['country_id_n']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# total_counts

### country_id_n by predicted classes

In [ ]:
grouped_counts = df.groupby(['country_id_n', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# grouped_counts

In [ ]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == 1]

# positive_grouped_counts

In [ ]:
merged_counts = pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

# merged_counts

### (DPPL) for country_id_n

In [ ]:
ppl_country_id_n = get_PPL(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count='predict_automatch'
)

ppl_country_id_n

In [ ]:
dppl_country_id_n = get_DPPL(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count='predict_automatch'
)

dppl_country_id_n

#### Bucketizing dppl_country_id_n

In [ ]:
print(pd.cut(dppl_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> Although it is likely. Basing solely on **DPPL** we can't assume if there is a disparity impact among **prod_gr_id** Classes yet. 

## Disparate (Adverse) Impact (DI)

![image info](./docs/images/DI.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

### CHARACTERISTICS OF METRIC:
> **DI >= 0**

> This measure may be considered fair if it resides in the **<0.8; 1.2>** range.

## (DI) for Variable: prod_gr_id

In [ ]:
ppl_prod_gr_id

In [ ]:
di_prod_gr_id = sorted([abs(i / j) for i in ppl_prod_gr_id for j in ppl_prod_gr_id if i != j])

#### Bucketizing di_prod_gr_id

In [ ]:
print(pd.cut(di_prod_gr_id, bins=4).value_counts())

#### CONCLUSIONS:

> For each possible relation between Classes the metric resides between **<0.8; 1.2>**

> We can loosely assume that there is no disparity impact between Classes of **prod_gr_id** Variable

## (DI) for Variable: country_id_n

In [ ]:
ppl_country_id_n

In [ ]:
di_country_id_n = sorted([abs(i / j) for i in ppl_country_id_n for j in ppl_country_id_n if i != j])

#### Bucketizing di_country_id_n

In [ ]:
print(pd.cut(di_country_id_n, bins=4).value_counts())

> Dividing Ratios by Ratios, so values close to 1 are good

In [ ]:
pd.Series(di_country_id_n).describe()

#### CONCLUSIONS:

> There is more than 100 Classes of Variable **country_id_n** out of **<0.8; 1.2>** bounds.

> At this moment we can loosely assume that there are some examples of Disparity Impact in case of **country_id_n** Variable.

## Difference in Conditional Outcome (DCO)
> Type 1: **Difference in Conditional Acceptance (DCA)**

> Type 2: **Difference in Conditional Rejection (DCR)**

> When both **DCA** and **DCR** are very close to **0**, we can conclude that the proportion of qualified (as suggested by observed labels) applicants accepted by the model and the proportion of unqualified applicants rejected are nearly equal across both classes.

### Difference in Conditional Acceptance (DCA)

![image info](./docs/images/DCA.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded

> Zero denominator is possible. In such case the allocations to each Class are too small and a warning should be issued.

## (DCA) for Variable: prod_gr_id

In [ ]:
ca_prod_gr_id = get_CA(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

ca_prod_gr_id

In [ ]:
dca_prod_gr_id = get_DCA(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dca_prod_gr_id

#### Bucketizing dca_prod_gr_id

In [ ]:
print(pd.cut(dca_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO: Diffrances are really small

## (DCA) for Variable: country_id_n

In [ ]:
ca_country_id_n = get_CA(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

ca_country_id_n

In [ ]:
df[df['country_id_n']=='109'].describe()

> from the CA and mean we can assume that the Model underperforms when 'country_id_n'=='109'

In [ ]:
dca_country_id_n = get_DCA(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dca_country_id_n

#### Bucketizing dca_country_id_n

In [ ]:
print(pd.cut(dca_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

### Difference in Conditional Rejection (DCR)

![image info](./docs/images/DCR.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded

> Zero denominator is possible. In such case the allocations to each Class are too small and a warning should be issued.

## (DCR) for Variable: prod_gr_id

In [ ]:
cr_prod_gr_id = get_CR(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

cr_prod_gr_id

In [ ]:
dcr_prod_gr_id = get_DCR(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dcr_prod_gr_id

#### Bucketizing dcr_prod_gr_id

In [ ]:
print(pd.cut(dcr_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (DCR) for Variable: country_id_n

In [ ]:
cr_country_id_n = get_CR(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

cr_country_id_n

In [ ]:
dcr_country_id_n = get_DCR(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch'
)

dcr_country_id_n

#### Bucketizing dcr_country_id_n

In [ ]:
print(pd.cut(dcr_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

In [ ]:
cm_by_class_prod_gr_id = get_class_cm(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

cm_by_class_prod_gr_id

In [ ]:
rd_prod_gr_id = get_RD(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

rd_prod_gr_id

#### Bucketizing rd_prod_gr_id

In [ ]:
print(pd.cut(rd_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (RD) for Variable: country_id_n

In [ ]:
cm_by_class_prod_gr_id = get_class_cm(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

cm_by_class_prod_gr_id

In [ ]:
recall_by_class = get_RD(
    data_frame=df,
    acctuals='class_acctual',
    predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

recall_by_class

#### Bucketizing recall_by_class

In [ ]:
print(pd.cut(recall_by_class, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## Difference in label rates (DLR)

![image info](./docs/images/DAR.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> DAR > -1 AND DAR < 1

> DRR > -1 AND DRR < 1

> DAR is the same as precision difference between the first and second classes.

## (DAR) for Variable: prod_gr_id

In [ ]:
ar_by_class_prod_gr_id = get_AR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

ar_by_class_prod_gr_id

In [ ]:
dar_by_class_prod_gr_id = get_DAR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

dar_by_class_prod_gr_id

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(dar_by_class_prod_gr_id, bins=3).value_counts())

## (DRR) for Variable: prod_gr_id

In [ ]:
rr_by_class_prod_gr_id = get_RR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

rr_by_class_prod_gr_id

In [ ]:
drr_by_class_prod_gr_id = get_DRR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

drr_by_class_prod_gr_id

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(drr_by_class_prod_gr_id, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (DAR) for Variable: country_id_n

In [ ]:
ar_by_class_country_id_n = get_AR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

ar_by_class_country_id_n

In [ ]:
dar_by_class_country_id_n = get_DAR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

dar_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(dar_by_class_country_id_n, bins=3).value_counts())

## (DRR) for Variable: country_id_n

In [ ]:
rr_by_class_country_id_n = get_RR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

rr_by_class_country_id_n

In [ ]:
drr_by_class_country_id_n = get_DRR(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

drr_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(drr_by_class_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## Accuracy DIfference (AD)

![image info](./docs/images/AD.jpg)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

#### CHARACTERISTICS OF METRIC:
> Unbounded metric

> Care needed when FN = 0

## (AD) for Variable: prod_gr_id

In [ ]:
a_by_class_prod_gr_id = get_A(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

rr_by_class_prod_gr_id

In [ ]:
ad_by_class_country_id_n = get_AD(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='prod_gr_id'
)

ad_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(ad_by_class_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO:

## (AD) for Variable: country_id_n

In [ ]:
a_by_class_country_id_n = get_A(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

rr_by_class_country_id_n

In [ ]:
ad_by_class_country_id_n = get_AD(
    data_frame=df,
    column_to_count_acctuals='class_acctual',
    column_to_count_predictions='predict_automatch',
    column_to_group_by='country_id_n'
)

ad_by_class_country_id_n

#### Bucketizing dar_by_class_prod_gr_id

In [ ]:
print(pd.cut(ad_by_class_country_id_n, bins=3).value_counts())

#### CONCLUSIONS:

> TODO: